In [ ]:
import os
os.chdir('../../..')

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
from examples.two_tank_system.dataset import TwoTankImgDataSet
from torch.utils.data import DataLoader

In [ ]:
dataset = TwoTankImgDataSet(debug=True)

In [ ]:
batch_size = 1000
dataloader = DataLoader(dataset, batch_size=batch_size, num_workers=24)

In [ ]:
from sindy_autoencoder_cps.lightning_module import SINDyAutoencoder
import yaml
# MODEL_VERSION = 'freq_and_phase'
MODEL_VERSION = 'version_40'

hparams_path = f'./lightning_logs/{MODEL_VERSION}/hparams.yaml'
with open(hparams_path, 'r') as stream:
        hparam_dct = yaml.safe_load(stream)
ckpt_file_name = os.listdir(f'./lightning_logs/{MODEL_VERSION}/checkpoints/')[-1]
ckpt_file_path = f'./lightning_logs/{MODEL_VERSION}/checkpoints/{ckpt_file_name}'
model = SINDyAutoencoder.load_from_checkpoint(ckpt_file_path)
model

In [ ]:
model.SINDyLibrary.device

In [ ]:
batches = iter(dataloader)

In [ ]:
batches.next()

In [ ]:
x, xdot, idxs = batches.next()
print(f'shape x: {x.shape},   shape xdot: {xdot.shape}')

In [ ]:
import numpy as np
x_2_norm = np.linalg.norm(x, axis=1)
x_2_norm_sum = x_2_norm.sum()
x_2_norm_sum

In [ ]:
xdot_2_norm = np.linalg.norm(xdot, axis=1)
xdot_2_norm_sum = xdot_2_norm.sum()
xdot_2_norm_sum

In [ ]:
x_2_norm_sum/xdot_2_norm_sum

In [ ]:
x_hat, xdot_hat, z, zdot, zdot_hat = model.cuda()(x.cuda(), xdot.cuda())

In [ ]:
import matplotlib.pyplot as plt
import examples.three_tank_system.constants as const
from ipywidgets import interact
import ipywidgets as widgets

def plot_x_space_fit_for_sample(sample=0):
    #select random sample and compare original and recostruction
    # sample = 477
    x_sample = x[sample, :].cpu().detach().numpy()
    xhat_sample = x_hat[sample, :].cpu().detach().numpy()
    z_sample = z[sample, :].cpu().detach().numpy()
    xdot_sample = xdot[sample, :].cpu().detach().numpy()
    xdot_hat_sample = xdot_hat[sample, :].cpu().detach().numpy()
    zdot_sample = zdot[sample, :].cpu().detach().numpy()
    zdot_hat_sample = zdot_hat[sample, :].cpu().detach().numpy()
    idx_sample = idxs[sample]
    f, axarr = plt.subplots(1,2)
    axarr[0].imshow(x_sample.reshape(const.PICTURE_SIZE, const.PICTURE_SIZE))
    axarr[1].imshow(xhat_sample.reshape(const.PICTURE_SIZE, const.PICTURE_SIZE))
    f, axarr = plt.subplots(1,2)
    axarr[0].imshow(xdot_sample.reshape(const.PICTURE_SIZE, const.PICTURE_SIZE))
    axarr[1].imshow(xdot_hat_sample.reshape(const.PICTURE_SIZE, const.PICTURE_SIZE))

interact(plot_x_space_fit_for_sample, sample=[25*i for i in range(100)])

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np


fig = make_subplots(rows=1, cols=3, shared_xaxes=False)

for i in range(3):
    fig.add_trace(go.Scatter(x=zdot[:, i].cpu().detach().numpy(),
                       y=zdot_hat[:, i].cpu().detach().numpy(),
                      mode='markers', name=f'zdot_hat{i} over zdot{i}',
#                              marker_color='blue',
                             opacity=.7), row=1, col=1+i)
fig.show()

In [ ]:
XI = model.XI.cpu().detach().numpy()
XI.max()

In [ ]:
import pandas as pd

In [ ]:
df_XI = pd.DataFrame(XI, columns=['z0_dot_hat', 'z1_dot_hat', 'z2_dot_hat'])
df_XI.index=model.SINDyLibrary.feature_names

In [ ]:
import plotly.express as px
print(model.SINDyLibrary.feature_names)
px.imshow(df_XI.abs())

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np

fig = make_subplots(rows=2, cols=3, shared_xaxes=True)

for i in range(3):
    fig.add_trace(go.Scatter(x=np.array(range(z.shape[0])),
                       y=z[:, i].cpu().detach().numpy(),
                      mode='markers', name=f'z{i}',
                             marker_color='blue', opacity=.5), row=1, col=1+i,
                    )
for i in range(3):
    fig.add_trace(go.Scatter(x=np.array(range(z.shape[0])),
                       y=zdot_hat[:, i].cpu().detach().numpy(),
                      mode='markers', name=f'zdot_hat{i}',
                            marker_color='blue', opacity=.4), row=2, col=1+i)
for i in range(3):
    fig.add_trace(go.Scatter(x=np.array(range(z.shape[0])),
                       y=zdot[:, i].cpu().detach().numpy(),
                      mode='markers', name=f'zdot{i}',
                             marker_color='green', opacity=.5), row=2, col=1+i)



fig.update_layout(height=600, width=1200, title_text="Side By Side Subplots")
fig.show()

In [ ]:
import examples.three_tank_system.constants as const
z_real = dataset.df.iloc[idxs][const.Z_COL_NAMES]
fig = make_subplots(rows=3, cols=3, shared_xaxes=True)

for i in range(3):
    for j in range(3):
        fig.add_trace(go.Scatter(y=z.detach().cpu().numpy()[:, i],
                           x=z_real.values[:, j],
                          mode='markers', name=f'z{i} over z_real{j}'), row=1+j, col=1+i)


fig.update_layout(height=600, width=1200, title_text="Real hidden state vs latent activations")
fig.show()

In [ ]:
zdot_real = dataset.df.iloc[idxs][const.Z_DOT_COL_NAMES]
fig = make_subplots(rows=3, cols=3, shared_xaxes=True)

for i in range(3):
    for j in range(3):
        fig.add_trace(go.Scatter(y=zdot.detach().cpu().numpy()[:, i],
                           x=zdot_real.values[:, j],
                          mode='markers', name=f'zdot{i} over zdot_real{j}'), row=1+j, col=1+i)


fig.update_layout(height=600, width=1200, title_text="Real hidden states derivatives vs predicted derivatives")
fig.show()

In [ ]:
model.XI_coefficient_mask